In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
from keras.preprocessing.image import ImageDataGenerator
import os

In [6]:
sz = 128

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [8]:
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
#classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
#classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2

In [9]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# categorical_crossentropy for more than 2

In [10]:
# Step 2 - Preparing the train/test data and training the model
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3686528   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

In [11]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
training_set = train_datagen.flow_from_directory('data2/train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

Found 9644 images belonging to 27 classes.


In [22]:
test_set = test_datagen.flow_from_directory('data2/test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

Found 3201 images belonging to 27 classes.


In [15]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=12845//10, # No of images in training set
        epochs=10,
        validation_data=test_set,
        validation_steps=3201//10)# No of images in test set

/Users/doddaballapurabhishek/miniforge3/envs/ml/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1284/1284 [==============================] - 39s 30ms/step - loss: 198.8499 - accuracy: 0.0366 - val_loss: 3.2957 - val_accuracy: 0.0392
Epoch 2/10
1284/1284 [==============================] - 39s 30ms/step - loss: 950.9420 - accuracy: 0.0394 - val_loss: 3.2955 - val_accuracy: 0.0390
Epoch 3/10
1284/1284 [==============================] - 39s 30ms/step - loss: 327.2739 - accuracy: 0.0371 - val_loss: 3.2954 - val_accuracy: 0.0390
Epoch 4/10
1284/1284 [==============================] - 39s 30ms/step - loss: 33.4973 - accuracy: 0.0348 - val_loss: 3.2955 - val_accuracy: 0.0390
Epoch 5/10
1284/1284 [==============================] - 38s 30ms/step - loss: 451.8897 - accuracy: 0.0398 - val_loss: 3.2954 - val_accuracy: 0.0392
Epoch 6/10
1284/1284 [==============================] - 38s 30ms/step - loss: 48.9846 - accuracy: 0.0373 - val_loss: 3.2954 - val_accuracy: 0.0390
Epoch 7/10
1284/1284 [==============================] - 40s 31ms/step - loss: 3.2954 - accuracy: 0.0430 - val_loss

In [16]:
# Saving the model
model_json = classifier.to_json()
with open("model-trail1-e10-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model-trail1-e10-bw.h5')
print('Weights saved')

Model Saved
Weights saved
